In [ ]:
import os
import glob

import pandas as pd
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
print(torchvision.__version__)
#from torchvision.io import read_image
from sys import getsizeof

In [ ]:
!cd /home/krom/projects/kaalge/easy_xray
!pwd
!ls 

In [ ]:
#train_df = pd.read_csv('train.csv')
train_metadata_df = pd.read_csv('dicom_metadata_train_merged.csv') # Took it from kaggle notebooks
#test_df = pd.read_csv('test.csv')
#train_df #.iloc[0,0]
import os.path
files_path = glob.glob("**/*.jpg",recursive=True)

#train_df['Target'].unique()
files_path
bone_files = [ f for f in files_path if f.endswith('bone.jpg')]
hsv_files = [ f for f in files_path if not f.endswith('bone.jpg')]
#hsv_files[3],"\n",train_df.iloc[666,0]#,bone_files
#train_df.loc['1.2.826.0.1.3680043.8.498.33785782871926445035243454630719127502']

for f in bone_files:
    if train_metadata_df['SOPInstanceUID'].iloc[666] in f:
        print(f, "\n", train_metadata_df['SOPInstanceUID'].iloc[666])
train_metadata_df.head(8).columns[64:]
#y.append(np.ndarray.flatten(np.array(train_merged_df.loc[train_merged_df['SOPInstanceUID'] == uid][train_merged_df.columns[64:]])))
    

In [ ]:
from torch.utils.data import Dataset
#from torchvision.io import read_image toooo old fot that stuff



class DicomDataset(Dataset):
    def __init__(self, merged_df, image_dir, bone = True, preload = True, transform=None, target_transform=None):
        self.img_labels = merged_df
        
        self.img_dir = image_dir
        self.transform = transform
        self.target_transform = target_transform
        self.files_path = glob.glob(image_dir+"/**/*.jpg",recursive=True)
        #print(len(self.img_labels), self.img_labels['SOPInstanceUID'])
        self.images = []
        self.file_pathes = []
        for im_num in range(len(self.img_labels)):
            item_name = self.img_labels['SOPInstanceUID'].iloc[im_num]
            #print(item_name)
            if bone:
                file_path = [ f for f in files_path if f.endswith(item_name + '-c.bone.jpg')]
            else:
                file_path = [ f for f in files_path if f.endswith(item_name + '-c.jpg')]
            self.file_pathes.append(file_path[0])
        #print(self.file_pathes)
        if preload:
            for img_path in self.file_pathes:
                
                #image = read_image(img_path) # my torch is too old. i'l use workaround
                image = Image.open(img_path)
                image = np.array(image).astype(np.uint8)
                self.images.append(image)
            self.preloaded = preload
        print(len(self.images))


    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        if self.preloaded:
            image = self.images[idx]
        else:
            image = read_image(self.file_pathes[idx])
        
        label = self.img_labels.iloc[idx, 64:]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    
    def get_all_data(self):
        return self.images[:10]


In [ ]:
from torchvision.transforms import ToTensor, ToPILImage
training_data = DicomDataset(
    merged_df=train_metadata_df,
    image_dir="train",
#    train=True,
#    download=True,
    transform=ToTensor()
)



In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import random_split

train_size = int(0.8 * len(training_data))
test_size = len(training_data) - train_size
train_dataset, test_dataset = random_split(training_data, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
train_dataloader
len(training_data)

In [ ]:
import torch as torch
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    print(img.shape)
    img = ToPILImage()(img.to('cpu'))
    figure.add_subplot(rows, cols, i)
    #plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img, cmap="gray")

plt.show()

In [ ]:
def compress(image, k):
    """
    Perform svd decomposition and truncate it (using k singular values/vectors)
    
    Parameters: 
        image (np.array):        input image (probably, colourful)
        
        k (int):                 approximation rank
        
    --------
    Returns:
      reconst_matrix (np.array): reconstructed matrix (tensor in colourful case)
      
      s (np.array):              array of singular values 
    """
    image2 = image.copy()
    
    Rf = image2[:,:,0] - image2[:,:,0].mean(axis=1, keepdims=True)
    Gf = image2[:,:,1] - image2[:,:,1].mean(axis=1, keepdims=True)
    Bf = image2[:,:,2] - image2[:,:,2].mean(axis=1, keepdims=True)
    
    # compute per-channel SVD for input image
    # <your code here>
    u_r, Rs, vh_r = np.linalg.svd(Rf, full_matrices=False)
    u_g, Gs, vh_g = np.linalg.svd(Gf, full_matrices=False)
    u_b, Bs, vh_b = np.linalg.svd(Bf, full_matrices=False)
    Rs = Rs[:k]
    Gs = Gs[:k]
    Bs = Bs[:k]
    
    # reconstruct the input image with the given approximation rank
    # <your code here>
    red_channel = u_r[:, :k] @ np.diag(Rs) @ vh_r[:k, :]
    green_channel = u_g[:, :k] @ np.diag(Gs) @ vh_g[:k, :]
    blue_channel = u_b[:, :k] @ np.diag(Bs) @ vh_b[:k, :]
    
    reduced_im = np.stack([red_channel, green_channel, blue_channel], axis=2)
    
    # save the array of top-k singular values
    s = np.zeros((len(Gs), 3))
    # <your code here>
    s[:, 0] = Rs
    s[:, 1] = Gs
    s[:, 2] = Bs
    
    return reduced_im.copy(), s


import torch as torch
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    #print(img.shape)
    img = ToPILImage()(img)
    img,sing_values = compress(np.asarray(img),20) ## You can try and change number here. Comparing qualty of image.
    #print(type(img), img.shape)
    img = img /  img.max() # unfortunately looks like compress rescale all date.
    figure.add_subplot(rows, cols, i)
    #plt.title(labels_map[label])
    plt.axis("off")
    #print(img[255])
    plt.imshow((img))

plt.show()

In [ ]:
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    print(img.shape)
    Rf = img[0,:,:] - img[0,:,:].mean()
    Gf = img[1,:,:] - img[1,:,:].mean()
    Bf = img[2,:,:] - img[2,:,:].mean()
    u, Rs, vh = np.linalg.svd(Rf, full_matrices=False)
    u, Gs, vh = np.linalg.svd(Gf, full_matrices=False)
    u, Bs, vh = np.linalg.svd(Bf, full_matrices=False)
    figure.add_subplot(rows, cols, i)
#plt.figure(figsize=(12,7))
    plt.plot(Rs,'ro')
    plt.plot(Gs,'g.')
    plt.plot(Bs,'b:')
    plt.yscale('log')
    plt.ylabel("Singular values")
    plt.xlabel("Singular value order")
plt.show()    

In [ ]:
from torchvision import models
alexnet = models.alexnet(pretrained=True)
alexnet.eval()




from torchvision import transforms

transform = transforms.Compose([            #[1]

 transforms.Resize(256),                    #[2]

 transforms.CenterCrop(224),                #[3]

 transforms.ToTensor()
])


img = Image.open("lot.png")
img_t = transform(ToPILImage()((training_data[1])[0].to('cpu')))
img_t = ((training_data[1])[0].to('cpu'))
batch_t = torch.unsqueeze(img_t, 0)
out = alexnet(batch_t)
#out = alexnet(train_dataloader)


In [ ]:
out.shape
all_data = training_data.get_all_data()
torch_all_data = torch.tensor(training_data.get_all_data())


In [ ]:
print(all_data[0].shape, torch_all_data.shape,training_data[0][0].shape )

In [ ]:
print(getsizeof(all_data) ,getsizeof(torch_all_data), torch_all_data.shape
)
shp = torch_all_data.shape
import torchvision.transforms as transforms
  
# Read a PIL image
  
# Define a transform to convert PIL 
# image to a Torch tensor
transforma = transforms.Compose([
    transforms.ToTensor()
])
#trans_base = transforms(np.array(all_data)) ValueError: pic should be 2/3 dimensional. Got 4 dimensions.
#trans = transforms(torch_all_data) TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>
reshape = torch_all_data.reshape(shp[0],shp[3],shp[1],shp[2])



In [ ]:
(reshape[0]/255.0) - training_data[0][0]
figure = plt.figure(figsize=(8, 8))
figure.add_subplot(3, 3, 1)
#plt.title(labels_map[label])
plt.axis("off")
    #print(img[255])
plt.imshow((transforms.ToPILImage()(reshape[0]/255.0)))

figure.add_subplot(3, 3, 2)
#plt.title(labels_map[label])
plt.axis("off")
    #print(img[255])
plt.imshow((transforms.ToPILImage()(training_data[0][0])))


figure.add_subplot(3, 3, 3)
#plt.title(labels_map[label])
plt.axis("off")
    #print(img[255])
plt.imshow((transforms.ToPILImage()(all_data[0])))

plt.show()
print(type(all_data[0]))
for i in all_data:
    i= transforms.ToTensor()(i)
all_data[3].shape

In [ ]:
import torch.nn as nn
n_features = 100
resnet = models.resnet18()
resnet.fc = nn.Linear(resnet.fc.in_features, n_features)
torch.save(resnet, 'model.pth')
model = torch.load('model.pth')



In [ ]:
#inp = torch.randn(20, 3, 224, 224)
#%time out = model(torch.tensor(training_data.get_all_data()))
print(torch.tensor(training_data.get_all_data()).shape)
#out

In [ ]:
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


In [ ]:
def select_features(X_train, y_train, X_test,i):
    # configure to select all features
    fs = SelectKBest(score_func=f_classif, k=i)
    # learn relationship from training data
    fs.fit(X_train, y_train)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs


In [ ]:
from sklearn.metrics import f1_score
import statistics
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split


x_train,x_test,y_train,y_test = train_test_split(features,labels,test_size=0.2)
x_train,x_test = select_features(x_train,y_train,x_test,275)


In [ ]:
from xgboost import XGBClassifier


classifier = XGBClassifier()
classifier.fit(x_train,y_train)
yhat = classifier.predict(x_test)
acc = accuracy_score(y_test, yhat)
print(confusion_matrix(y_test, yhat))
Y_test = y_test
Y_pred = yhat
l = confusion_matrix(Y_test, Y_pred)

print(f"\nAccuracy score : ", accuracy_score(Y_test, Y_pred))
print(f"\nconfusion matrix : \n", confusion_matrix(Y_test, Y_pred))
print(f"\nf1 score : ", f1_score(Y_test, Y_pred, average="macro"))
print(f"\nprecision score : ", precision_score(Y_test, Y_pred, average="macro"))
print(f"\nSpecifity : ", (l[1,1]+l[2,2]) / (l[1,1]+l[2,2]+l[1,0]+l[2,0]))
print(f"\nSensitivity : ", l[0,0] / (l[0,0] + l[0,1]+l[0,2]))
print(f"\n============")